In [11]:
import constraint_programming
import re
import math

WORDS = "../words1.txt"
CROSSWORD = "../crossword1.1.txt"

In [12]:
def read_words(path):
    f = open(path, "r")
    lines = f.readlines()
    return [line.strip() for line in lines]

words = read_words(WORDS)
print(words)

['to', 'at', 'tea', 'eat']


In [13]:
hashes_regex = re.compile("\.+")

def parse_horizontally(path):
    f = open(path, "r")
    horizontal_dots = []
    lines = f.readlines()
    for i, line in enumerate(lines):
        line = line.strip()
        positions = hashes_regex.finditer(line)
        for pos in positions:
            dot_lines = ((pos.span()[0], i), (pos.span()[1]-1, i))
            horizontal_dots.append(dot_lines)
    return horizontal_dots

def get_columns(path):
    f = open(path, "r")
    lines = f.readlines()
    columns = ['' for i in range(len(lines))]
    for line in lines:
        for caract in enumerate(line):
            if caract[1] != "\n":
                columns[caract[0]] += caract[1]
    return columns
    
# NB : the y axis is reversed
def parse_vertically(path):
    vertical_dots = []
    columns = get_columns(path)
    for i, column in enumerate(columns):
        positions = hashes_regex.finditer(column)
        for pos in positions:
            dot_columns = ((i, pos.span()[0]), (i, pos.span()[1]-1))
            vertical_dots.append(dot_columns)
    return vertical_dots

def remove_single_dots(dots):
    no_single_dots = []
    for dot in dots:
        d = math.fabs(dot[0][0]-dot[1][0]) + math.fabs(dot[0][1]-dot[1][1])
        if d>1:
            no_single_dots.append(dot)
    return no_single_dots
        
        
def read_crossword(path):
    horizontal_dots = parse_horizontally(path)
    horizontal_dots = remove_single_dots(horizontal_dots)
    vertical_dots = parse_vertically(path)
    vertical_dots = remove_single_dots(vertical_dots)
    dots = {"horizontal": horizontal_dots, "vertical": vertical_dots}
    return dots
    
lines = read_crossword(CROSSWORD)
print(lines)

{'horizontal': [((1, 0), (3, 0))], 'vertical': [((2, 0), (2, 3))]}


In [14]:
def find_intersection(h_line, v_line):
    # Better: use the fact that it is necessary for the intersection to have v_line x position and h_line y position
    a = h_line[0][0]<= v_line[0][0]
    b = v_line[0][0]<=h_line[1][0]
    c = v_line[0][1]<=h_line[0][1]
    d = v_line[0][1]<= h_line[0][1]
    e = h_line[0][1]<=v_line[1][1]
    f = v_line[0][0]>=h_line[0][0]
    if a and b and c and d and e and f:
        return (v_line[0][0], h_line[0][1])
    else:
        return None

def find_intersections(lines):
    intersections = []
    """
    [
        {
            "which": (
                nieme ligne de dots["horizontal"],
                mieme colonne de dots["vertical"]
                ),
            "position": (x, y)
    ]
    """
    for i, h_line in enumerate(lines["horizontal"]):
        for j, v_line in enumerate(lines["vertical"]):
            intersection = find_intersection(h_line, v_line)
            if intersection != None:
                intersections.append({"position":intersection, "which":(i, j)})
    return intersections

intersections = find_intersections(lines)
print(intersections)

[{'position': (2, 0), 'which': (0, 0)}]
